In [19]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import pickle
import os

In [20]:
timitFile = "./features/test_mfcc/timit.hdf"
energyFlag = 0
directoryNameOfSavedModel = "./models/mfcc_4_with_energy_coeff/"

In [21]:
timit_df = pd.read_hdf(timitFile)
timit_df.head()

,features,labels
0,"[7.71423780751174, -19.035095285088897, -17.90...",sil
1,"[7.688952486328098, -17.370392040822527, -13.3...",sil
2,"[7.564117679250088, -19.8182882602723, -15.955...",sil
3,"[6.9988543168974084, -16.020609949203447, -4.6...",sil
4,"[6.73811699753896, -18.012102593354562, -7.778...",sil


In [22]:
features = np.array(timit_df["features"].tolist())
labels = np.array(timit_df["labels"].tolist())
print("Features array shape: ",features.shape)
print("Labels: ",list(set(labels)))

Features array shape:  (451660, 13)
Labels:  ['', 'ow', 'y', 'ey', 'w', 'b', 'ih', 'g', 'd', 'iy', 'sil', 'eh', 'r', 'p', 's', 'th', 'er', 'aw', 'ah', 'n', 'ch', 'm', 'l', 'f', 'uh', 'v', 'jh', 'ng', 'z', 'sh', 'k', 'oy', 'ay', 'uw', 'ae', 'dx', 'aa', 't', 'hh', 'dh']


In [23]:
if energyFlag == 1:
    numberOfFeatures,numberOfDimensions = features.shape
    if numberOfDimensions==13:
        requiredDimensions = list(range(13))
        requiredDimensions.pop(0)
        features = features[:, requiredDimensions]
    elif numberOfDimensions==26:
        requiredDimensions = list(range(26))
        requiredDimensions.pop(0)
        requiredDimensions.pop(14)
        features = features[:, requiredDimensions]
    elif numberOfDimensions==39:
        requiredDimensions = list(range(39))
        requiredDimensions.pop(0)
        requiredDimensions.pop(14)
        requiredDimensions.pop(27)
        features = features[:, requiredDimensions]
print("Features array shape: ",features.shape)

Features array shape:  (451660, 13)


In [24]:
allGMMs=[]
allFiles=[]
for root, dirs, files in os.walk(directoryNameOfSavedModel):
    for filename in files:
        allFiles.append(filename)
        f = open(directoryNameOfSavedModel+filename, 'rb')
        GMM = pickle.load(f)
        allGMMs.append(GMM)
        f.close()
print("GMMs of all phonemes: ",allFiles)

allScores = []
for GMM in allGMMs:
    allScores.append(GMM.score_samples(features))
print("Scores of each GMM with respect to all phonemes: ",allScores)

predictedLabels=[]
for i in range(0,len(features)):
    featureScores = [j[i] for j in allScores]
    maxScore = featureScores[0]
    maxIndex = 0
    for i in range(1,len(featureScores)):
        if(featureScores[i] > maxScore):
            maxScore = featureScores[i]
            maxIndex = i
    if str(allFiles[maxIndex]) == "BLANK.pkl":
        predictedLabels.append('')
    else:
        predictedLabels.append(str(allFiles[maxIndex])[:-4])
print("Original labels",labels)
print("Predicted labels",predictedLabels)

frameLevelAccuracy = 0.0
correctPredictions = 0
totalPredictions = len(predictedLabels)
for i in range(0,len(predictedLabels)):
    if(predictedLabels[i] == labels[i]):
        correctPredictions = correctPredictions+1
frameLevelAccuracy = correctPredictions/totalPredictions
print("Frame level accuracy in %: ", frameLevelAccuracy*100)

GMMs of all phonemes:  ['uw.pkl', 'oy.pkl', 'ae.pkl', 'k.pkl', 'y.pkl', 'n.pkl', 'l.pkl', 'aa.pkl', 'aw.pkl', 'z.pkl', 'ch.pkl', 'm.pkl', 'hh.pkl', 'ey.pkl', 'dh.pkl', 'iy.pkl', 'BLANK.pkl', 'sil.pkl', 'eh.pkl', 'ih.pkl', 'dx.pkl', 'jh.pkl', 'er.pkl', 'sh.pkl', 'w.pkl', 'uh.pkl', 'v.pkl', 't.pkl', 'ay.pkl', 'b.pkl', 'f.pkl', 'ow.pkl', 'g.pkl', 'p.pkl', 'r.pkl', 'ah.pkl', 'ng.pkl', 'th.pkl', 'd.pkl', 's.pkl']
Scores of each GMM with respect to all phonemes:  [array([-47.09389868, -45.89500201, -49.49099053, ..., -50.50955984,
       -51.61981145, -52.67889943]), array([-48.81319634, -49.10304792, -52.42009904, ..., -52.57888036,
       -53.67993219, -54.41872151]), array([-46.75740337, -46.26649652, -49.59725407, ..., -50.60124431,
       -51.39003461, -52.68110532]), array([-51.25350726, -48.89956199, -51.24371637, ..., -56.92543147,
       -57.9444546 , -58.83249017]), array([-61.46693131, -60.61439286, -62.48689666, ..., -61.21395271,
       -60.83647368, -61.73760086]), array([-47.1

Frame level accuracy in %:  14.030465394323164


In [25]:
import asr_evaluation_modified_14

print("Phoneme error rate in %: ",asr_evaluation_modified_14.main(labels,predictedLabels))

Phoneme error rate in %:  0.8865274853907299
